# Gaussian PRM test

In [ ]:
import pickle
import numpy as np

from swarm_prm.utils.gaussian_prm import GaussianPRM
from swarm_prm.utils.gaussian_utils import GaussianGraphNode
from swarm_prm.envs.obstacle_map import ObstacleMap, Obstacle 

In [ ]:

roadmap = ObstacleMap(200, 160)
safety_radius = 2 
pos = np.array([50, 50])
obstacles  = [
                Obstacle(None, "POLYGON", [(50, 160), (50, 80), (70, 80), (70, 160)]),
                Obstacle(None, "POLYGON", [(70, 160), (70, 120), (150, 120), (150, 160)]),
                Obstacle(None, "POLYGON", [(50, 40), (50, 0), (130, 0), (130, 40)]),
                Obstacle(None, "POLYGON", [(130, 80), (130, 0), (150, 0), (150, 80)]),
                Obstacle(None, "POLYGON", [(95, 100), (95, 80), (105, 80), (110, 100)]),
                Obstacle(None, "POLYGON", [(95, 80), (90, 60), (105, 60), (105, 80)]),
]

num_samples = 200 

for obs in obstacles:
    roadmap.add_obstacle(obs)
# fig, ax = roadmap.visualize()
start_mean = np.random.rand(2) * 100
goal_mean = np.random.rand(2) * 100

# Agent config
num_agent = 100
agent_radius = 2
sampling_method = "GAUSSIAN_SAMPLING"
#sampling_method = "SWARMPRM_SAMPLING"
hex_radius = 3
TEST_CVAR = False

gaussian_prm = GaussianPRM(obstacle_map=roadmap, num_samples=num_samples)

gaussian_prm.sample_free_space(sampling_strategy="CVT")
gaussian_prm.build_roadmap(roadmap_method="TRIANGULATION")

gaussian_prm.build_roadmap(roadmap_method="TRIANGULATION", radius=50, collision_check_method="CVAR")
fig, ax = gaussian_prm.visualize_roadmap("test_map")
gaussian_prm.visualize_g_nodes()

with open("solutions/gaussian_prm.pkl", "wb") as f:
    pickle.dump(gaussian_prm, f)


In [ ]:
from shapely.geometry import Polygon
start_region = [Polygon([[0, 160], [0, 0], [50, 0], [50, 160]])]
goal_region = [Polygon([[150, 160], [150, 0], [200, 0], [200, 160]])]

start_indices = gaussian_prm.get_node_index(start_region)
goal_indices = gaussian_prm.get_node_index(goal_region)

fig, ax = gaussian_prm.visualize_g_nodes()

# Randomly picking starts and goals

num_starts = 20
num_goals = 20

starts = np.random.choice(start_indices, num_starts)
goals = np.random.choice(goal_indices, num_goals)

# Verify start capacity and goal capacity

start_capacity = 0
for start in starts:
    start_capacity += gaussian_prm.gaussian_nodes[start].get_capacity(agent_radius)

goal_capacity = 0
for goal in goals:
    goal_capacity += gaussian_prm.gaussian_nodes[goal].get_capacity(agent_radius)

assert start_capacity > num_agent, "start config smaller than required."
assert goal_capacity > num_agent, "goal config smaller than required."


for node_idx in starts:
    gaussian_prm.gaussian_nodes[node_idx].visualize(ax)

for node_idx in goals:
    gaussian_prm.gaussian_nodes[node_idx].visualize(ax, edgecolor="b")

